In [1]:
import torch 
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
import os
warnings.filterwarnings("ignore")
print(torch.cuda.is_available())

True


In [2]:
from transformers import AutoTokenizer, LlamaForCausalLM
import torch
cache_dir = '/shared/s1/lab06/PLMs/Asclepius_7B/models--starmpcc--Asclepius-7B/snapshots/9207fa5ec28658219040b890a522516273ec9a7b'
tokenizer = AutoTokenizer.from_pretrained(cache_dir)
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#model = LlamaForCausalLM.from_pretrained(cache_dir).to(device)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)
model = AutoModelForCausalLM.from_pretrained(
    cache_dir,
    quantization_config=bnb_config, # Use bitsandbytes config
    device_map="auto",  # Specifying device_map="auto" so that HF Accelerate will determine which GPU to put each layer of the model on
    trust_remote_code=True, # Set trust_remote_code=True to use falcon-7b model with custom code
)

tokenizer = AutoTokenizer.from_pretrained(cache_dir, trust_remote_code=True) # Set trust_remote_code=True
tokenizer.pad_token = tokenizer.eos_token # Setting pad_token same as eos_token


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [41]:
prompt = """You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
{question}
[question End]

"""

question = "What is the diagnosis?"


In [42]:
def generate_answer(question):
    model_input = prompt.format(question=question)
    input_ids = tokenizer(model_input, return_tensors="pt").input_ids
    output = model.generate(input_ids, max_length = 2000)
    print(tokenizer.decode(output[0]))   

### Q325

In [11]:
question = "Acrodermatitis enteropathica, Birk-Landau-Perez syndrome, and Spondylocheirodysplastic Ehlers-Danlos syndrome are genetic conditions that all involve abnormalities related to metabolism of what metal? 1.Copper 2.Iron 3.Magnesium, 4.Zinc"
model_input = prompt.format(question=question)
input_ids = tokenizer(model_input, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length = 1000)
print(tokenizer.decode(output[0]))

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene.
Write an answer with detailed explanation.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Acrodermatitis enteropathica, Birk-Landau-Perez syndrome, and Spondylocheirodysplastic Ehlers-Danlos syndrome are genetic conditions that all involve abnormalities related to metabolism of what metal? 1.Copper 2.Iron 3.Magnesium, 4.Zinc
[question End]

Based on the given information, the genetic conditions mentioned in the question involve abnormalities related to the metabolism of copper. Specifically, Acrodermatitis enteropathica, Birk-Landau-Perez syndrome, and Spondylocheirodysplastic Ehlers-Danlos syndrome all involve copper metabolism abnormalities.</s><s>


### Q326

In [16]:
question = "Which of the following would represent a frameshift variant? 1. c.169_170insA, 2. c.79_80delinsTT, 3. g.123A>G, 4. p.Trp24Cys"

In [17]:
model_input = prompt.format(question=question)
input_ids = tokenizer(model_input, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length = 1000)
print(tokenizer.decode(output[0]))

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene.
Write an answer with detailed explanation.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Which of the following would represent a frameshift variant? 1. c.169_170insA, 2. c.79_80delinsTT, 3. g.123A>G, 4. p.Trp24Cys
[question End]

The variant c.169_170insA would represent a frameshift variant as it involves a change in the nucleotide sequence, resulting in a shift in the reading frame and potentially leading to a change in the amino acid sequence. The variants c.79_80delinsTT and g.123A>G also involve changes in the nucleotide sequence, but they do not represent frameshift variants as they do not result in a shift in the reading frame. The variant p.Trp24Cys is a missense variant, which involves a change in the amino acid sequence, but it is not a frameshift variant.</s><s>


### q327


In [21]:
###q327
question = "cc contribute to a number of different human diseases, including in complex/multifactorial way."
generate_answer(question)

'<s> You are an intelligent gene-related question expert.\nBelow is a question related to the gene.\nWrite an answer with detailed explanation.\nThe response should provide an accurate answer to the question, while being concise.\n\n[Question begin]\ncc contribute to a number of different human diseases, including in complex/multifactorial way.\n[question End]\n\nYes, it is possible that the CC genes contribute to a number of different human diseases in a complex/multifactorial way. However, without further information on the specific genes and diseases involved, it is difficult to provide a more specific answer.</s><s>'

### 328

In [23]:
question = "On examination, a patient is noted to have “coarse” facial features.  Based on this finding, which type of testing would most likely be indicated? 1. Chromosomal breakage, 2. FISH for 22q11.2, 3. FMR1 repeat analysis, 4. Urine glycosaminoglycans"
generate_answer(question)

'<s> You are an intelligent gene-related question expert.\nBelow is a question related to the gene.\nWrite an answer with detailed explanation.\nThe response should provide an accurate answer to the question, while being concise.\n\n[Question begin]\nOn examination, a patient is noted to have “coarse” facial features.  Based on this finding, which type of testing would most likely be indicated? 1. Chromosomal breakage, 2. FISH for 22q11.2, 3. FMR1 repeat analysis, 4. Urine glycosaminoglycans\n[question End]\n\nBased on the information provided, the type of testing that would most likely be indicated is Chromosomal breakage. This is because the patient was noted to have "coarse" facial features, which could potentially indicate a genetic disorder resulting from chromosomal abnormalities. However, it is important to note that further testing may be necessary to confirm this suspicion.</s><s>'

### q329

In [32]:
question = "A man and a woman have 2 children, both of whom are affected by an autosomal recessive condition. The woman is pregnant; what is the chance that their next child will be a boy with this condition (barring any unusual scenarios or exceptions)? 0.025% 12.5% 25% 33%" 
generate_answer(question)

'<s> You are an intelligent gene-related question expert.\nBelow is a question related to the gene.\nWrite an answer with detailed explanation.\nThe response should provide an accurate answer to the question, while being concise.\n\n[Question begin]\nA man and a woman have 2 children, both of whom are affected by an autosomal recessive condition. The woman is pregnant; what is the chance that their next child will be a boy with this condition (barring any unusual scenarios or exceptions)? 0.025% 12.5% 25% 33%\n[question End]\n\nThe chance of having a boy with the autosomal recessive condition is 0.025% or 1 in 400. This is based on the information provided in the question.</s><s>'

### Q330

In [39]:
question = "An infant was slightly premature and is being taken care of in the NICU; on rounds, the team notes “puffiness” under the eyes, and that the calcium level has been high on routine blood chemistries.  What genetic condition might be suspected based on this? 22q11.2 deletion syndrome Homocystinuria Marfan syndrome Williams syndrome"
generate_answer(question)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene.
Write an answer with detailed explanation.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
An infant was slightly premature and is being taken care of in the NICU; on rounds, the team notes “puffiness” under the eyes, and that the calcium level has been high on routine blood chemistries.  What genetic condition might be suspected based on this? 22q11.2 deletion syndrome Homocystinuria Marfan syndrome Williams syndrome
[question End]

Based on the information provided, 22q11.2 deletion syndrome is the most likely genetic condition that could be suspected for the premature infant with puffiness under the eyes and high calcium levels. However, further testing would be needed to confirm this suspicion.</s><s>


### Q331

In [53]:
question = "(Choose only one option) A patient has features consistent with Turner syndrome. Assuming this is the correct diagnosis, what result would be expected on routine cytogenomic studies (with the caveat that mosaic results are not uncommon)? 45,X 45,XO 46,XX 47,XXX"
generate_answer(question)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene.
Write an answer with detailed explanation.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
(Choose only one option) A patient has features consistent with Turner syndrome. Assuming this is the correct diagnosis, what result would be expected on routine cytogenomic studies (with the caveat that mosaic results are not uncommon)? 45,X 45,XO 46,XX 47,XXX
[question End]

Turner syndrome is a genetic condition that results in the complete or partial absence of the X chromosome. Therefore, the expected result on routine cytogenomic studies for a patient with Turner syndrome would be 45,X or 45,XO, indicating the complete or partial absence of the X chromosome. However, it is important to note that mosaic results are not uncommon in cytogenomic studies, so the actual result may not be as straightforward as expected.</s><s>


### Q332

In [54]:
question = "Specific genetic changes from genetic testing can help guide therapy. In a patient with certain types of cancer, identification of a germline pathogenic variant in BRCA1 or BRCA2 might lead to consideration of therapy with what type of agent? 1.AAV9-based gene therapy 2.Nitrosourea 3.PARP inhibitor 4.PD-1 inhibitor"

generate_answer(question)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene.
Write an answer with detailed explanation.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Specific genetic changes from genetic testing can help guide therapy. In a patient with certain types of cancer, identification of a germline pathogenic variant in BRCA1 or BRCA2 might lead to consideration of therapy with what type of agent? 1.AAV9-based gene therapy 2.Nitrosourea 3.PARP inhibitor 4.PD-1 inhibitor
[question End]

Based on the given information, the patient's germline pathogenic variant in BRCA1 or BRCA2 may suggest consideration of PARP inhibitor therapy. However, it is important to note that there are other factors that may need to be taken into account when deciding on the appropriate therapy, such as the patient's overall health and the specific type and stage of cancer.</s><s>


### Q333

In [57]:
question = "Which of the following is a membrane-bound organelle that contains digestive enzymes and is involved in a number of cellular processes, including apoptosis, metabolism of large molecules made by the body, and destruction of pathogens? Golgi body Lysosome Mitochondrion Ribosome"
generate_answer(question)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene.
Write an answer with detailed explanation.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Which of the following is a membrane-bound organelle that contains digestive enzymes and is involved in a number of cellular processes, including apoptosis, metabolism of large molecules made by the body, and destruction of pathogens? Golgi body Lysosome Mitochondrion Ribosome
[question End]

The organelle that contains digestive enzymes and is involved in a number of cellular processes including apoptosis, metabolism of large molecules made by the body, and destruction of pathogens is the lysosome.</s><s>


### Q334

In [66]:
q = "An infant has a positive newborn screen for X-linked adrenoleukodystrophy (X-ALD). Follow-up tests are negative for X-ALD, but the child starts to show signs of developmental delay in infancy. What condition should be considered? 1.Aarskog syndrome 2.Aicardi-Goutières syndrome 3.Alagille syndrome 4.Alpha thalassemia"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should answer something.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
An infant has a positive newborn screen for X-linked adrenoleukodystrophy (X-ALD). Follow-up tests are negative for X-ALD, but the child starts to show signs of developmental delay in infancy. What condition should be considered? 1.Aarskog syndrome 2.Aicardi-Goutières syndrome 3.Alagille syndrome 4.Alpha thalassemia
[question End]

Based on the given information, it is unclear which condition the child should be considered to have. The follow-up tests for X-ALD were negative, but the child showed signs of developmental delay in infancy. Therefore, it is possible that the child has a different genetic condition that causes developmental delay, but further testing would be nee

### Q335

In [68]:
q ="During a viral pandemic, genetic variants that similarly increase transmissibility (and therefore result in a selective advantage) appear independently in different geographic locations. What is the best term for this? Convergent evolution Divergent evolution Mutualism Parsimony"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should answer something.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
During a viral pandemic, genetic variants that similarly increase transmissibility (and therefore result in a selective advantage) appear independently in different geographic locations. What is the best term for this? Convergent evolution Divergent evolution Mutualism Parsimony
[question End]

The best term for this is convergent evolution, as multiple genetic variants that increase transmissibility appear independently in different geographic locations during a viral pandemic.</s><s>


### Q336

In [69]:
q = "A patient has undergone extensive genetic testing without finding an explanation. They enroll in a research study involving a new type of genomic analysis. As part of enrollment, informed consent is obtained. What basic ethical principle includes informed consent? Beneficence, Justice, Respect for persons, Subject selection"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should answer something.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A patient has undergone extensive genetic testing without finding an explanation. They enroll in a research study involving a new type of genomic analysis. As part of enrollment, informed consent is obtained. What basic ethical principle includes informed consent? Beneficence, Justice, Respect for persons, Subject selection
[question End]

The basic ethical principle that includes informed consent is "Respect for persons." This principle requires that individuals be informed about the risks, benefits, and alternatives related to their medical care, and that they provide their informed consent before any treatment is administered.</s><s>


### Q337

In [73]:
q = "A two year-old girl has been diagnosed with Rett syndrome. In addition to supportive and other care, what type of surveillance is recommended based on this diagnosis? Choose only one answer from the following: Colonoscopy, Electrocardiogram, LH and FSH levels, Melanoma screening"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should answer something.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A two year-old girl has been diagnosed with Rett syndrome. In addition to supportive and other care, what type of surveillance is recommended based on this diagnosis? Choose only one answer from the following: Colonoscopy, Electrocardiogram, LH and FSH levels, Melanoma screening
[question End]

Based on the diagnosis of Rett syndrome, it is recommended to perform regular colonoscopies to monitor for any potential colon issues. Additionally, it may be beneficial to monitor electrocardiogram levels and perform regular skin checks for melanoma. However, there is no information provided in the given discharge summary about the recommended frequency or timing of these tests. It i

### Q338

In [77]:
q = "A patient has features consistent with Noonan syndrome. If this is the correct diagnosis, which gene is most likely to be involved? KRAS NF1 PTPN11 RAF1"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A patient has features consistent with Noonan syndrome. If this is the correct diagnosis, which gene is most likely to be involved? KRAS NF1 PTPN11 RAF1
[question End]

Based on the given information, it is unclear which gene is most likely to be involved in Noonan syndrome. The discharge summary does not specify which gene is associated with this condition. Further testing and evaluation may be necessary to determine the specific gene involved in Noonan syndrome.</s><s>


### Q339

In [78]:
q = "On physical exam, a child is noted to have unilateral leukocoria (the pupillary reflex appears white during the eye exam). Which condition might be suspected? Macular degeneration Retinitis pigmentosa Retinoblastoma Tay-Sachs disease"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
On physical exam, a child is noted to have unilateral leukocoria (the pupillary reflex appears white during the eye exam). Which condition might be suspected? Macular degeneration Retinitis pigmentosa Retinoblastoma Tay-Sachs disease
[question End]

Based on the given information, the condition that might be suspected is Retinitis pigmentosa, as it is a genetic disorder that affects the retina and can cause vision loss. However, further testing and evaluation may be needed to confirm the diagnosis.</s><s>


### Q342

In [80]:
q = "The first form of gene therapy to be FDA-approved for use in the United States, treats what condition? Acute lymphoblastic leukemia Cystic fibrosis Leber congenital amaurosis Sickle cell anemia"

generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
The first form of gene therapy to be FDA-approved for use in the United States, treats what condition? Acute lymphoblastic leukemia Cystic fibrosis Leber congenital amaurosis Sickle cell anemia
[question End]

The first form of gene therapy to be FDA-approved for use in the United States treats cystic fibrosis.</s><s>


### Q344

In [82]:
q = "An apparently healthy woman gives birth to a term male infant. Her brother had a severe inborn error of metabolism, and testing shows that her newborn son is also affected. Assuming no consanguinity or other such circumstance, what condition is most likely in this family? Biotinidase deficiency OTC deficiency Phenylketonuria Wilson disease"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
An apparently healthy woman gives birth to a term male infant. Her brother had a severe inborn error of metabolism, and testing shows that her newborn son is also affected. Assuming no consanguinity or other such circumstance, what condition is most likely in this family? Biotinidase deficiency OTC deficiency Phenylketonuria Wilson disease
[question End]

Based on the information provided, the most likely condition in this family is Biotinidase deficiency. This is because the newborn son is affected and Biotinidase deficiency is the only condition mentioned as being present in the woman's brother.</s><s>


### Q345

In [87]:
q = "A patient has clinical features involved in 22q11.2 deletion syndrome but microarray is normal. Which gene might be involved? 1.ELN 2.PAFAH1B1 3.RAI1 4.TBX1"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A patient has clinical features involved in 22q11.2 deletion syndrome but microarray is normal. Which gene might be involved? 1.ELN 2.PAFAH1B1 3.RAI1 4.TBX1
[question End]

Based on the information provided, it is unclear which gene may be involved in the patient's condition as the microarray is normal and there is no mention of any specific genes that were tested. Further testing and analysis may be necessary to determine which gene, if any, is involved in the patient's condition.</s><s>


### Q346

In [86]:
q = " A patient undergoes genetic testing because of a strong family history of papillary renal cell carcinoma. Which of the following genes would most likely be indicated for inclusion in a genetic testing panel for this patient? 1.ANKRD26 2.MET 3.RET 4.STK11"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
 A patient undergoes genetic testing because of a strong family history of papillary renal cell carcinoma. Which of the following genes would most likely be indicated for inclusion in a genetic testing panel for this patient? 1.ANKRD26 2.MET 3.RET 4.STK11
[question End]

Based on the given information, the gene most likely to be indicated for inclusion in a genetic testing panel for this patient is RET. This is because papillary renal cell carcinoma is associated with mutations in the RET gene, and including this gene in a genetic testing panel would be the most appropriate way to identify any potential genetic predisposition to the diseas

### Q347

In [88]:
q = "Prenatal ultrasound shows findings consistent with Thanataphoric dysplasia; molecular testing via amniocentesis confirms the diagnosis. No parental testing has been done yet. What is the most likely recurrence risk? a. <1% b. 25% c. 33% d. 50%"

generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Prenatal ultrasound shows findings consistent with Thanataphoric dysplasia; molecular testing via amniocentesis confirms the diagnosis. No parental testing has been done yet. What is the most likely recurrence risk? a. <1% b. 25% c. 33% d. 50%
[question End]

The most likely recurrence risk for Thanataphoric dysplasia is 50%, as it is an autosomal recessive disorder. However, since no parental testing has been done yet, it is impossible to determine the recurrence risk for the parents without further genetic testing.</s><s>


### Q348

In [89]:
q ="A patient is suspected to have mosaicism for a specific genetic variant (a rare single nucleotide variant). Which of the following assays would likely provide the most precise information about the level of mosaicism for this variant? 1.Digital droplet PCR 2.Genome sequencing 3.SNP microarray 4.NGS-based panel testing"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A patient is suspected to have mosaicism for a specific genetic variant (a rare single nucleotide variant). Which of the following assays would likely provide the most precise information about the level of mosaicism for this variant? 1.Digital droplet PCR 2.Genome sequencing 3.SNP microarray 4.NGS-based panel testing
[question End]

Based on the information provided, the assay that would likely provide the most precise information about the level of mosaicism for the genetic variant is SNP microarray. This is because it can detect the copy number variation (deletion or duplication) associated with the genetic variant, which can provide mo

### Q349

In [90]:
q = "Genetics is consulted because a young patient has a hand malformation and ipsilateral pectoralis major hypoplasia. Which term refers to these findings? Klippel-Feil anomaly Pectus carinatum Poland anomaly Sprengel anomaly"
generate_answer(q)


<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Genetics is consulted because a young patient has a hand malformation and ipsilateral pectoralis major hypoplasia. Which term refers to these findings? Klippel-Feil anomaly Pectus carinatum Poland anomaly Sprengel anomaly
[question End]

Klippel-Feil anomaly is the term that refers to the young patient's hand malformation and ipsilateral pectoralis major hypoplasia.</s><s>


### Q350

In [6]:
q = "Due to family history and genetic testing in affected relatives, a healthy patient is suspected to have a balanced translocation. Which test would be most likely to help efficiently identify this translocation? 1.Karyotype 2.NGS-based panel testing 3.Sanger sequencing 4.SNP-based microarray"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Due to family history and genetic testing in affected relatives, a healthy patient is suspected to have a balanced translocation. Which test would be most likely to help efficiently identify this translocation? 1.Karyotype 2.NGS-based panel testing 3.Sanger sequencing 4.SNP-based microarray
[question End]

Based on the information provided, karyotype testing would be the most efficient method for identifying the balanced translocation in the healthy patient. However, if the patient has a rare genetic condition or the translocation involves a large number of genes, NGS-based panel testing or Sanger sequencing may be necessary. SNP-based mic

### Q351

In [7]:
q = "A pathogenic variant in which of the following genes can cause a phenotype similar to deletion of chromosome 17p11.2? 1.NF1 2.RAI1 3.RBM8A 4.TBX1"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A pathogenic variant in which of the following genes can cause a phenotype similar to deletion of chromosome 17p11.2? 1.NF1 2.RAI1 3.RBM8A 4.TBX1
[question End]

Based on the given information, it is not possible to determine which gene has a pathogenic variant that can cause a phenotype similar to deletion of chromosome 17p11.2, as there are multiple genes listed as potential answers. Further testing and evaluation would be required to determine the specific gene with the pathogenic variant.</s><s>


### Q352

In [9]:
q = "A patient has a family history that is highly suspicious for Brugada syndrome. Assuming the clinical suspicion is correct, what gene is most likely to be involved? 1.CACNA1C 2.SCN1B 3.SCN5A 4.SEMA3A"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A patient has a family history that is highly suspicious for Brugada syndrome. Assuming the clinical suspicion is correct, what gene is most likely to be involved? 1.CACNA1C 2.SCN1B 3.SCN5A 4.SEMA3A
[question End]

Based on the family history and clinical suspicion, it is most likely that Brugada syndrome is caused by a mutation in one of the following genes: CACNA1C, SCN1B, SCN5A, or SEMA3A. However, further genetic testing would be necessary to confirm which gene is involved.</s><s>


### Q353

In [11]:
q = "A man and a woman are seen together for prenatal counseling. The couple, both of whom are healthy, are from a community where a childhood-onset autosomal recessive genetic condition affects about 1 in 100 people. The man’s brother has the condition. What is the approximate chance that the couple’s child will be affected? 0.1% 1% 3% 6%"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A man and a woman are seen together for prenatal counseling. The couple, both of whom are healthy, are from a community where a childhood-onset autosomal recessive genetic condition affects about 1 in 100 people. The man’s brother has the condition. What is the approximate chance that the couple’s child will be affected? 0.1% 1% 3% 6%
[question End]

The approximate chance that the couple's child will be affected by the genetic condition is 0.1%. This is because the condition affects 1 in 100 people in the community, and the couple is both healthy. The chance of the child being affected increases if one or both parents have the condition, 

### Q354

In [14]:
q = "A patient with neonatal-onset epilepsy has been found to have bi-allelic pathogenic variants in the gene ALDH7A1. What treatment might be effective based on the genetic information? Allopurinol Colchicine Levodopa Pyridoxine"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A patient with neonatal-onset epilepsy has been found to have bi-allelic pathogenic variants in the gene ALDH7A1. What treatment might be effective based on the genetic information? Allopurinol Colchicine Levodopa Pyridoxine
[question End]

Based on the genetic information, allopurinol may be effective for the patient as it has been shown to reduce seizure frequency in patients with ALDH7A1 deficiency. However, further studies are needed to confirm the efficacy of allopurinol in this specific patient. Colchicine and Levodopa may also be considered as they have been used in patients with ALDH7A1 deficiency. Pyridoxine may also be considered

### Q355

In [23]:
q = "A patient seen by genetics is noted to have developmental delay, obesity, postaxial polydactyly, and renal disease. What condition is most likely? 1.Bardet-Biedl syndrome 2.Joubert syndrome 3.McKusick-Kaufman syndrome 4.Senior-Loken syndrome"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A patient seen by genetics is noted to have developmental delay, obesity, postaxial polydactyly, and renal disease. What condition is most likely? 1.Bardet-Biedl syndrome 2.Joubert syndrome 3.McKusick-Kaufman syndrome 4.Senior-Loken syndrome
[question End]

Based on the given information, the patient's symptoms are most likely associated with Bardet-Biedl syndrome. This is a rare genetic disorder that affects multiple body systems and can result in a range of symptoms including obesity, developmental delay, and renal disease. However, it is important to note that without further testing and evaluation, a definitive diagnosis cannot be made

### Q356

In [22]:
q = "An apparently healthy adult woman is referred to genetics because her father and sister have what appears to be a type of familial hypertrophic cardiomyopathy. Who would be the most informative family member to test first? 1.The woman’s child 2.The woman’s father 3.The woman herself  4.The woman’s mother"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
An apparently healthy adult woman is referred to genetics because her father and sister have what appears to be a type of familial hypertrophic cardiomyopathy. Who would be the most informative family member to test first? 1.The woman’s child 2.The woman’s father 3.The woman herself  4.The woman’s mother
[question End]

The woman's child would be the most informative family member to test first as they would inherit the genetic mutation from the mother.</s><s>


### Q357

In [25]:
q = "An apparently healthy adult woman was referred to genetics because her father and sister have familial hypertrophic cardiomyopathy. Genetic testing reveals a VUS in a relevant gene in the father, which is not resolved by further familial testing. Based on this, what is recommended for the woman? 1.Discharge from care 2.Genetic testing of the woman’s children 3.Immediate ICD placement 4.Regular clinical cardiac evaluation"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
An apparently healthy adult woman was referred to genetics because her father and sister have familial hypertrophic cardiomyopathy. Genetic testing reveals a VUS in a relevant gene in the father, which is not resolved by further familial testing. Based on this, what is recommended for the woman? 1.Discharge from care 2.Genetic testing of the woman’s children 3.Immediate ICD placement 4.Regular clinical cardiac evaluation
[question End]

Based on the information provided, it is recommended that the woman undergo genetic testing of her children. However, it is unclear if the VUS found in the father is related to the familial hypertrophic car

### Q358

In [27]:
q = "A several month-old boy has a scaly skin rash. Assuming a genetic diagnosis, deficiency of what enzyme might be suspected? 1.Aldolase B 2.Arginase 3.Mevalonate kinase 4.Steroid sulfatase"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A several month-old boy has a scaly skin rash. Assuming a genetic diagnosis, deficiency of what enzyme might be suspected? 1.Aldolase B 2.Arginase 3.Mevalonate kinase 4.Steroid sulfatase
[question End]

Based on the given information, deficiency of aldolase B is suspected in the several month-old boy with a scaly skin rash. This is because aldolase B is responsible for the conversion of fatty acids into energy, and its deficiency can lead to a buildup of fatty acids in the body, which can cause a variety of symptoms including a scaly skin rash.</s><s>


### Q359

In [29]:
q = "Which of the following refers to the most common isochromosome in humans? i(1q) i(18p) i(Xq) i(Yp)"
generate_answer(q)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Which of the following refers to the most common isochromosome in humans? i(1q) i(18p) i(Xq) i(Yp)
[question End]

The most common isochromosome in humans is i(1q), which is a result of the duplication of the short arm of chromosome 1.</s><s>


### Q361

In [34]:
question = "The Genetic Information Nondiscrimination Act (GINA) of 2008 aims to protect Americans from discrimination based on their genetic information related to which of the choices below?"

choices = [
    "1.Disability insurance and Life insurance",
    "2.Employment",
    "3.Health insurance and Employment",
    "4.Long-term care insurance"
]

query = question + "\n" + "\n".join(choices)

In [33]:
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
The Genetic Information Nondiscrimination Act (GINA) of 2008 aims to protect Americans from discrimination based on their genetic information related to which of the choices below?
1.Disability insurance and Life insurance
2.Employment
3.Health insurance and Employment
4.Long-term care insurance
[question End]

The Genetic Information Nondiscrimination Act (GINA) of 2008 aims to protect Americans from discrimination based on their genetic information related to their health insurance and employment. However, it does not specify which of the choices provided in the question is the correct answer. Therefore, it is unanswerable with the given

### 363

In [37]:
question = "A woman is affected by an autosomal dominant condition (that is completely penetrant). She has three children. What is the chance that all three of her children are affected by the condition?"

choices = [
"12.5%",
"25%",
"50%",
"75%"\
]
query = question + "\n" + "\n".join(choices)

In [39]:
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A woman is affected by an autosomal dominant condition (that is completely penetrant). She has three children. What is the chance that all three of her children are affected by the condition?
12.5%
25%
50%
75%
[question End]

The chance that all three of her children are affected by the condition is 25%.</s><s>


### Q364

In [40]:
question = "Due to physical examination and family history findings suggestive of a genetic condition, a person has been found to have a pathogenic variant in FBN1. Based on this, what medical treatment would most likely be indicated (though this would only be indicated for some conditions related to this gene)?"
choices = [
"Anakinra",
"Beta blocker",
"Carnitine",
"Digoxin"
]
query = question + "\n" + "\n".join(choices)

In [41]:
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose at least one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Due to physical examination and family history findings suggestive of a genetic condition, a person has been found to have a pathogenic variant in FBN1. Based on this, what medical treatment would most likely be indicated (though this would only be indicated for some conditions related to this gene)?
Anakinra
Beta blocker
Carnitine
Digoxin
[question End]

Based on the information provided, the medical treatment most likely indicated for the patient would be Anakinra, a medication used to treat a rare genetic disorder called Neurofibromatosis type 1 (NF1). However, it is important to note that this is only a likely indication for some condi

### Q365

In [15]:
question = "Based on medical history and examination, a patient is suspected to have Pallister-Killian syndrome. What tissue source(s) or sample type(s) would likely be most helpful to confirm the diagnosis?"
choices = [
"1. CSF",
"2. Peripheral blood or saliva",
"3. Buccal swab or skin biopsy",
"4. Urine"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Based on medical history and examination, a patient is suspected to have Pallister-Killian syndrome. What tissue source(s) or sample type(s) would likely be most helpful to confirm the diagnosis?
1. CSF
2. Peripheral blood or saliva
3. Buccal swab or skin biopsy
4. Urine
[question End]

Based on the given information, CSF would likely be the most helpful tissue source or sample type to confirm the diagnosis of Pallister-Killian syndrome. However, it is important to note that additional information about the patient's medical history and examination may be necessary to determine the most appropriate sample type or tissue source for confirming this d

### Q366

In [14]:
question = "For which condition is there an FDA-approved gene therapy available?"
choices = [
"1. ADA-SCID",
"2. Familial hypercholesterolemia",
"3. Progeria",
"4. Spinal muscular atrophy"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
For which condition is there an FDA-approved gene therapy available?
1. ADA-SCID
2. Familial hypercholesterolemia
3. Progeria
4. Spinal muscular atrophy
[question End]

There is currently an FDA-approved gene therapy available for ADA-SCID, which is a form of severe combined immune deficiency caused by a mutation in the ADA gene. However, there are no FDA-approved gene therapies available for the other conditions listed in the question, including familial hypercholesterolemia, progeria, and spinal muscular atrophy.</s><s>


### Q367

In [16]:
question = "Gene therapies have been developed and tested to treat a number of genetic conditions, including a type of spinal muscular atrophy (SMA), which was FDA-approved in 2019. Gene therapy and gene editing-based treatments are being studied related to many other conditions. Of note, gene therapy for ADA-SCID has been approved in Europe, but not by the FDA in the US."
choices = [
"1. Euchromatin",
"2. Haplotype",
"3. Pseudogene",
"4. Recombinant"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Gene therapies have been developed and tested to treat a number of genetic conditions, including a type of spinal muscular atrophy (SMA), which was FDA-approved in 2019. Gene therapy and gene editing-based treatments are being studied related to many other conditions. Of note, gene therapy for ADA-SCID has been approved in Europe, but not by the FDA in the US.
1. Euchromatin
2. Haplotype
3. Pseudogene
4. Recombinant
[question End]

Euchromatin refers to the chromosomal DNA that is rich in Cytosine-Adenine (CA) repeat sequences. Haplotype refers to a group of genes that are inherited together as a unit. Pseudogene refers to a gene that has a similar

### 369

In [18]:
question = "A child with optic atrophy has been diagnosed with Wolfram syndrome. Testing for what other clinical manifestation would be warranted?"
choices = [
"1. Diabetes mellitus",
"2. Hirschsprung disease",
"3. Hypercholesterolemia",
"4. Immunodeficiency"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A child with optic atrophy has been diagnosed with Wolfram syndrome. Testing for what other clinical manifestation would be warranted?
1. Diabetes mellitus
2. Hirschsprung disease
3. Hypercholesterolemia
4. Immunodeficiency
[question End]

Based on the given information, it is not clear what other clinical manifestation would be warranted for testing. The discharge summary only mentions optic atrophy and Wolfram syndrome. There is no information provided about other potential clinical manifestations that would require testing. Therefore, it is not possible to answer this question with the given information.</s><s>


### 370

In [19]:
question = "A neonate has an older brother with molecularly-confirmed familial hemophagocytic lymphohistiocytosis (fHLH). Based on the causes of fHLH, what is the most likely chance that the neonate will also be “genetically affected” (have also inherited the causative variant(s))?"
choices = [
"1. 25%",
"2. 33%",
"3. 50%",
"4. 67%"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A neonate has an older brother with molecularly-confirmed familial hemophagocytic lymphohistiocytosis (fHLH). Based on the causes of fHLH, what is the most likely chance that the neonate will also be “genetically affected” (have also inherited the causative variant(s))?
1. 25%
2. 33%
3. 50%
4. 67%
[question End]

Based on the information provided, the neonate has a 50% chance of being genetically affected by familial hemophagocytic lymphohistiocytosis (fHLH).</s><s>


### 371

In [20]:
question = "Overall in humans, autosomal aneuploidies most commonly occur due to events during which stage of meiosis?"
choices = [
"1. Maternal meiosis I",
"2. Maternal meiosis II",
"3. Paternal meiosis I%",
"4. Paternal meiosis II%"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Overall in humans, autosomal aneuploidies most commonly occur due to events during which stage of meiosis?
1. Maternal meiosis I
2. Maternal meiosis II
3. Paternal meiosis I%
4. Paternal meiosis II%
[question End]

Based on the information provided, it is not possible to determine the stage of meiosis during which autosomal aneuploidies most commonly occur in humans. The discharge summary does not specify which stage of meiosis is most commonly associated with the occurrence of autosomal aneuploidies in humans.</s><s>


### 373

In [23]:
question = "You are seeing a patient suspected to have septo-optic dysplasia. In addition to agenesis/anomalies of the septum pellucidum and corpus callosum and optic nerve hypoplasia, what finding would you most likely anticipate?"
choices = [
"1. Hirschsprung disease",
"2. Melanoma",
"3. Pituitary anomalies",
"4. Rhizomelia"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
You are seeing a patient suspected to have septo-optic dysplasia. In addition to agenesis/anomalies of the septum pellucidum and corpus callosum and optic nerve hypoplasia, what finding would you most likely anticipate?
1. Hirschsprung disease
2. Melanoma
3. Pituitary anomalies
4. Rhizomelia
[question End]

I would anticipate pituitary anomalies as the most likely finding in a patient suspected to have septo-optic dysplasia.</s><s>


### 374

In [26]:
question = "A woman and a man have a child. Later, the man has another child, this time with the woman’s sister. Approximately what proportion of their DNA would the two children be expected to have in common?"
choices = [
"1. 25%",
"2. 37.5%",
"3. 50%",
"4. 67%"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A woman and a man have a child. Later, the man has another child, this time with the woman’s sister. Approximately what proportion of their DNA would the two children be expected to have in common?
1. 25%
2. 37.5%
3. 50%
4. 67%
[question End]

The two children would be expected to share approximately 37.5% of their DNA in common, as the man and woman had a child together, and then the man had another child with the woman's sister.</s><s>


### 375

In [27]:
question = "You are asked to do a genetics consult for a patient whose condition includes overgrowth, vascular malformations, and skin findings (eg, epidermal nevus). Genome sequencing on peripheral blood was unrevealing. Which gene might you suspect?"
choices = [
"1. GNAQ",
"2. PIK3CA",
"3. PTPN11",
"4. TP53"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
You are asked to do a genetics consult for a patient whose condition includes overgrowth, vascular malformations, and skin findings (eg, epidermal nevus). Genome sequencing on peripheral blood was unrevealing. Which gene might you suspect?
1. GNAQ
2. PIK3CA
3. PTPN11
4. TP53
[question End]

Based on the information provided, I would suspect GNAQ as a potential gene candidate for the patient'<s>


### 377

In [28]:
question = "An adult woman has a history of easy bruising and prolonged bleeding. Her father had acute myelogenous leukemia; she mentions that he also seemed to have similar bruising and bleeding issues. What gene would most likely be involved?"
choices = [
"1. PTEN",
"2. PTCH1",
"3. RUNX1",
"4. VHL"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
An adult woman has a history of easy bruising and prolonged bleeding. Her father had acute myelogenous leukemia; she mentions that he also seemed to have similar bruising and bleeding issues. What gene would most likely be involved?
1. PTEN
2. PTCH1
3. RUNX1
4. VHL
[question End]

Based on the information provided, the gene most likely involved in the patient's easy bruising and prolonged bleeding is VHL. This is because VHL is associated with hereditary hemorrhagic telangiectasia (HHT), which is a genetic disorder that causes abnormalities in blood vessels and can lead to easy bruising and prolonged bleeding. However, it is important to note that 

### 379

In [30]:
question = "A genetic condition that affects about 1 in 50,000 infants is added to newborn screening. The assay for the condition has 99.9% sensitivity and 99.9% specificity. An infant has a positive screen result. Based on just this information, what is the (approximate) chance the child has the condition?"
choices = [
"1. 0.01%",
"2. 2%",
"3. 98%",
"4. 99.9%"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A genetic condition that affects about 1 in 50,000 infants is added to newborn screening. The assay for the condition has 99.9% sensitivity and 99.9% specificity. An infant has a positive screen result. Based on just this information, what is the (approximate) chance the child has the condition?
1. 0.01%
2. 2%
3. 98%
4. 99.9%
[question End]

Based on the information provided, the approximate chance that the child has the genetic condition is 99.9%.</s><s>


### 380

In [32]:
question = "A man and a woman both have a clinical condition because they both have a certain heterozygous genetic variant. They have a child, who inherits both genetic changes (one from each parent). The child is much more severely affected than the parents. What is the term for this inheritance?"
choices = [
"1. Codominance",
"2. Incomplete dominance",
"3. Variable expressivity",
"4. X-linked inheritance"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A man and a woman both have a clinical condition because they both have a certain heterozygous genetic variant. They have a child, who inherits both genetic changes (one from each parent). The child is much more severely affected than the parents. What is the term for this inheritance?
1. Codominance
2. Incomplete dominance
3. Variable expressivity
4. X-linked inheritance
[question End]

The term for this inheritance is codominance.</s><s>


### 382

In [33]:
question = "A person has a very rare genetic condition that affects about 1 in 500,000 people. A test to detect this condition has 95% sensitivity and 99% specificity. What is the chance that the test will be negative for this individual?"
choices = [
"1. 1%",
"2. 5%",
"3. 95%",
"4. 99%"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A person has a very rare genetic condition that affects about 1 in 500,000 people. A test to detect this condition has 95% sensitivity and 99% specificity. What is the chance that the test will be negative for this individual?
1. 1%
2. 5%
3. 95%
4. 99%
[question End]

The chance that the test will be negative for this individual is 99%.</s><s>


### 383

In [34]:
question = "A person undergoes enzymatic testing for beta-hexosaminidase A (HEX A, related to Tay-Sachs disease). The result shows absent enzyme activity. However, the person is not affected, and does not carry a disease-associated allele. What is the best explanation for this?"
choices = [
"1. Allele dropout",
"2. Gene duplication",
"3. Nondisjunction"
"4. Pseudodeficiency"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A person undergoes enzymatic testing for beta-hexosaminidase A (HEX A, related to Tay-Sachs disease). The result shows absent enzyme activity. However, the person is not affected, and does not carry a disease-associated allele. What is the best explanation for this?
1. Allele dropout
2. Gene duplication
3. Nondisjunction4. Pseudodeficiency
[question End]

The best explanation for the absence of enzyme activity in the patient is allele dropout, which means that the enzyme gene is not present or is not functional. This can happen due to a variety of reasons, such as gene deletion or mutation. However, in this case, the patient does not carry a diseas

### 384

In [36]:
question = "Some genetic conditions have considerable genetic heterogeneity (the condition can be caused by variants in different genes). Which of the following conditions, all of which can involve predisposition to certain types of cancer, involves the most genetic heterogeneity?"
choices = [
"1. Fanconi anemia",
"2. Li-Fraumeni syndrome",
"3. Tuberous sclerosis",
"4. Von Hippel-Lindau syndrome"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Some genetic conditions have considerable genetic heterogeneity (the condition can be caused by variants in different genes). Which of the following conditions, all of which can involve predisposition to certain types of cancer, involves the most genetic heterogeneity?
1. Fanconi anemia
2. Li-Fraumeni syndrome
3. Tuberous sclerosis
4. Von Hippel-Lindau syndrome
[question End]

Based on the information provided, Von Hippel-Lindau syndrome involves the most genetic heterogeneity in predisposition to certain types of cancer. This is because the condition is caused by a mutation in the VHL gene, which can cause tumors in multiple organs and tissues. Fa

In [37]:
question = "You order a microarray for a child with a neurobehavorial phenotype involving autism and developmental delay; he has no other apparent pertinent clinical findings. Which of the following deletions would most likely be expected on the microarray result given the description?"
choices = [
"1. 16p11.2",
"2. 17q11.2",
"3. 7q11.23",
"4. 8q23.3-q24.11"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
You order a microarray for a child with a neurobehavorial phenotype involving autism and developmental delay; he has no other apparent pertinent clinical findings. Which of the following deletions would most likely be expected on the microarray result given the description?
1. 16p11.2
2. 17q11.2
3. 7q11.23
4. 8q23.3-q24.11
[question End]

Based on the given information, the most likely deletion on the microarray result for the child with neurobehavorial phenotype involving autism and developmental delay would be 7q11.23, as this is the most common deletion associated with autism. However, it is important to note that the given information does not 

### 387

In [39]:
question = "A woman has been diagnosed with breast cancer. Which of the following findings would most suggest that this could be related to a pathogenic variant in BRCA1 or BRCA2?"
choices = [
"1. No affected relatives",
"2. Onset at 80 years of age",
"3. Triple-negative status",
"4. Unilateral cancer"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent gene-related question expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A woman has been diagnosed with breast cancer. Which of the following findings would most suggest that this could be related to a pathogenic variant in BRCA1 or BRCA2?
1. No affected relatives
2. Onset at 80 years of age
3. Triple-negative status
4. Unilateral cancer
[question End]

Based on the given information, the finding of unilateral cancer would be most suggestive of a pathogenic variant in BRCA1 or BRCA2. However, it is important to note that other factors such as family history and genetic testing results should also be considered in assessing the likelihood of a BRCA1 or BRCA2 diagnosis.</s><s>


### 388

In [43]:
question = "A person undergoes exome sequencing. Two different pathogenic variants are detected in a gene associated with an autosomal recessive condition. Which of the following would likely best determine the variants’ inheritance (to see if the variants are in cis or trans)?"
choices = [
"1. Genome sequencing",
"2. Karyotype",
"3. Microarray testing",
"4. Parental testing"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A person undergoes exome sequencing. Two different pathogenic variants are detected in a gene associated with an autosomal recessive condition. Which of the following would likely best determine the variants’ inheritance (to see if the variants are in cis or trans)?
1. Genome sequencing
2. Karyotype
3. Microarray testing
4. Parental testing
[question End]

Based on the information provided, it is not possible to determine the inheritance of the pathogenic variants without further genetic testing. The exome sequencing only detects the presence of the variants, not their mode of inheritance. Additional testing, such as genome sequencing or karyotype, would be requ

### 390

In [44]:
question = "Based on clinical history and previous testing, you suspect that a patient has ornithine transcarbamylase (OTC) deficiency. What biochemical test might help confirm this diagnosis?"
choices = [
"1. Blood succinylacetone",
"2. CSF guanidinoacetate ",
"3. Total carnitine",
"4. Urine orotic acid"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Based on clinical history and previous testing, you suspect that a patient has ornithine transcarbamylase (OTC) deficiency. What biochemical test might help confirm this diagnosis?
1. Blood succinylacetone
2. CSF guanidinoacetate 
3. Total carnitine
4. Urine orotic acid
[question End]

Based on the given information, the biochemical test that can help confirm the diagnosis of ornithine transcarbamylase deficiency is urine orotic acid.</s><s>


### 391

In [45]:
question = "A man is affected by an extremely rare autosomal recessive genetic condition. What is the chance that the man’s grandchild will be a carrier of the condition? "
choices = [
"1. 12.5%",
"2. 25%",
"3. 50%",
"4. 100%"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A man is affected by an extremely rare autosomal recessive genetic condition. What is the chance that the man’s grandchild will be a carrier of the condition? 
1. 12.5%
2. 25%
3. 50%
4. 100%
[question End]

The chance that the man's grandchild will be a carrier of the condition is not explicitly stated in the given information. However, if the man has an autosomal recessive genetic condition, there is a high probability that his grandchild will also carry the condition. The likelihood of the grandchild being a carrier increases with the number of affected family members.</s><s>


### 392

In [48]:
question = "You suspect that your patient has myotonic dystrophy. Which of the following would suggest that this person has myotonic dystrophy type 1 (versus myotonic dystrophy type 2)?"
choices = [
"1. Arrhythmias",
"2. Cataracts",
"3. Developmental delay",
"4. Myotonia"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
You suspect that your patient has myotonic dystrophy. Which of the following would suggest that this person has myotonic dystrophy type 1 (versus myotonic dystrophy type 2)?
1. Arrhythmias
2. Cataracts
3. Developmental delay
4. Myotonia
[question End]

Based on the given information, the presence of arrhythmias would suggest that the patient has myotonic dystrophy type 1, as arrhythmias are a common symptom of this type of myotonic dystrophy. However, the presence of cataracts and developmental delay do not provide a clear indication of the type of myotonic dystrophy, as they can also occur in both types of the disease. The only symptom that can clearly differen

### 393

In [49]:
question = "You follow a young adult who has been diagnosed with Lynch syndrome. Her father is also affected; he was diagnosed with colorectal cancer in his late 30s. According to current guidelines, at about what age is colonoscopy recommended for your patient?"
choices = [
"1. 10-15 years of age",
"2. 20-25 years of age",
"3. 30-35 years of age",
"4. Around 50 years of age"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
You follow a young adult who has been diagnosed with Lynch syndrome. Her father is also affected; he was diagnosed with colorectal cancer in his late 30s. According to current guidelines, at about what age is colonoscopy recommended for your patient?
1. 10-15 years of age
2. 20-25 years of age
3. 30-35 years of age
4. Around 50 years of age
[question End]

Based on the given information, it is recommended that colonoscopy be performed when a patient is 20-25 years of age for individuals with Lynch syndrome who have a family history of colorectal cancer. However, it is important to note that this recommendation may vary depending on the patient's individual risk 

### 394

In [50]:
question = "A young adult has anomalies affecting the limbs, but extensive testing, exam, and history do not reveal any other medical issues, such as affecting other organ systems, or neurodevelopmental issues. What gene would most likely be involved?"
choices = [
"1. BBS1",
"2. FANCA",
"3. GDF5",
"4. NIPBL"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A young adult has anomalies affecting the limbs, but extensive testing, exam, and history do not reveal any other medical issues, such as affecting other organ systems, or neurodevelopmental issues. What gene would most likely be involved?
1. BBS1
2. FANCA
3. GDF5
4. NIPBL
[question End]

Based on the information provided, the gene most likely involved in the limb anomalies is BBS1. However, it is important to note that additional testing and examination may be necessary to confirm this diagnosis.</s><s>


### 395

In [52]:
question = "You are designing a laboratory assay to test for a condition that involves a “repeat expansion” (related to a repeat disorder, like Fragile X syndrome, Huntington disease, or Myotonic dystrophy type 1). Which of the following lab techniques would most likely describe the main test methodology that would be used for this assay?"
choices = [
"1. FISH testing",
"2. PCR fragment analysis",
"3. Sanger sequencing",
"4. SNP-based microarray"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
You are designing a laboratory assay to test for a condition that involves a “repeat expansion” (related to a repeat disorder, like Fragile X syndrome, Huntington disease, or Myotonic dystrophy type 1). Which of the following lab techniques would most likely describe the main test methodology that would be used for this assay?
1. FISH testing
2. PCR fragment analysis
3. Sanger sequencing
4. SNP-based microarray
[question End]

Based on the information provided, the main test methodology that would likely be used for this assay is FISH testing. This is because FISH testing is commonly used to detect repeat expansions, and it is a relatively simple and inexpensive

### Q396

In [57]:
question = "A woman undergoes standard non-invasive prenatal screening (NIPS) to assess for certain genetic conditions affecting the fetus. What incidental finding would be most likely to be identified (and potentially reported) via this test?"
choices = [
"1. Consanguinity",
"2. Fragile X syndrome",
"3. Maternal cancer",
"4. Tuberculosis"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A woman undergoes standard non-invasive prenatal screening (NIPS) to assess for certain genetic conditions affecting the fetus. What incidental finding would be most likely to be identified (and potentially reported) via this test?
1. Consanguinity
2. Fragile X syndrome
3. Maternal cancer
4. Tuberculosis
[question End]

The most likely incidental finding identified via non-invasive prenatal screening would be consanguinity, which refers to the relationship between the parents and the genetic makeup of the fetus. Fragile X syndrome, maternal cancer, and tuberculosis are other potential findings that could be identified through this test.</s><s>


### 397

In [59]:
q = "Multiple United States Supreme Court cases have involved decisions that directly or indirectly involved biomedical patent questions. Which case most directly led to the current stance that human genes cannot be patented?"
choices = [
"1. AMP v. Myriad",
"2. Diamond v. Chakrabarty",
"3. LabCorp v. Metabolite",
"4. Mayo v. Prometheus"
]
query = question + "\n" + "\n".join(choices)
generate_answer(q)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Multiple United States Supreme Court cases have involved decisions that directly or indirectly involved biomedical patent questions. Which case most directly led to the current stance that human genes cannot be patented?
[question End]

The case that most directly led to the current stance that human genes cannot be patented is the 2013 Supreme Court decision in Association for Molecular Pathology v. Myriad Genetics, which found that genes are not patentable subject matter.</s><s>


### 398

In [60]:
question = "A patient is suspected to have a genetic cause of rhabdomyolysis. A condition affecting which pathway or mechanism would be most likely?"
choices = [
"1. Bile acid synthesis",
"2. Fatty acid oxidation",
"3. Peroxisomal metabolism",
"4. Urea cycle"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A patient is suspected to have a genetic cause of rhabdomyolysis. A condition affecting which pathway or mechanism would be most likely?
1. Bile acid synthesis
2. Fatty acid oxidation
3. Peroxisomal metabolism
4. Urea cycle
[question End]

Based on the information provided, the patient's suspected genetic cause of rhabdomyolysis would most likely affect the peroxisomal metabolism pathway. This is supported by the fact that rhabdomyolysis is a rare complication of peroxisomal disorders, and the patient's clinical presentation of muscle breakdown and dark urine. However, further genetic testing and evaluation would be necessary to confirm this suspicion.</s><s>


### 399

In [61]:
question = "In a person with sickle cell disease, which treatment would be instituted to help reduce vaso-occlusive sequelae, including pain episodes and acute chest syndrome?"
choices = [
"1. Hydroxyurea",
"2. Penicillin",
"3. Pneumococcal vaccine",
"4. TNF inhibitors"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
In a person with sickle cell disease, which treatment would be instituted to help reduce vaso-occlusive sequelae, including pain episodes and acute chest syndrome?
1. Hydroxyurea
2. Penicillin
3. Pneumococcal vaccine
4. TNF inhibitors
[question End]

Based on the given information, the treatment that would be instituted to help reduce vaso-occlusive sequelae in a person with sickle cell disease is Hydroxyurea. This medication helps to prevent blood clots and reduces the frequency of pain episodes. However, it is important to note that the given information does not provide details on the specific dosage or duration of treatment.</s><s>


### 400

In [62]:
question = "A couple has several children who appear to be affected by a genetic condition. Cytogenetic testing via peripheral blood reveals a cause in one of the parents, both of whom are healthy. Which of the following is the most likely finding in the parent?"
choices = [
"1. Monosomy",
"2. Paracentric inversion",
"3. Pericentric inversion",
"4. Trisomy"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A couple has several children who appear to be affected by a genetic condition. Cytogenetic testing via peripheral blood reveals a cause in one of the parents, both of whom are healthy. Which of the following is the most likely finding in the parent?
1. Monosomy
2. Paracentric inversion
3. Pericentric inversion
4. Trisomy
[question End]

Based on the information provided, the most likely finding in the parent is trisomy. This is because the genetic condition appears to be inherited in an autosomal recessive pattern, which requires the presence of two copies of the mutated gene, one from each parent. Monosomy and paracentric/pericentric inversions are less likely

### 401

In [63]:
question ="A person has a suspected genetic condition, but exome testing does not identify the cause. Subsequent genome sequencing does identify the cause, however. Based on this, what type of variant was most likely identified by genome sequencing?"
choices = [
"1. Deep intronic variant",
"2. Exonic variant",
"3. Mosaic variant",
"4. Multi-gene deletion"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A person has a suspected genetic condition, but exome testing does not identify the cause. Subsequent genome sequencing does identify the cause, however. Based on this, what type of variant was most likely identified by genome sequencing?
1. Deep intronic variant
2. Exonic variant
3. Mosaic variant
4. Multi-gene deletion
[question End]

Based on the information provided, the most likely variant identified by genome sequencing was a deep intronic variant. This is because exome testing is known to miss a significant percentage of the genetic variants, especially the ones that affect introns. However, it is possible that there were other variants identified as well

### 403

In [64]:
question = "The mitochondrial genome contains multiple genes. Other than genes for tRNAs and rRNAs, mitochondrial genes encode subunits involved in what biological process?"
choices = [
"1. Glycolysis",
"2. Oxidative phosphorylation",
"3. Photosynthesis",
"4. Ubiquitylation"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
The mitochondrial genome contains multiple genes. Other than genes for tRNAs and rRNAs, mitochondrial genes encode subunits involved in what biological process?
1. Glycolysis
2. Oxidative phosphorylation
3. Photosynthesis
4. Ubiquitylation
[question End]

The mitochondrial genome encodes subunits involved in oxidative phosphorylation, which is the biological process that produces ATP.</s><s>


### 405

In [66]:
question = "At his annual check-up, a young adult mentions a strong family history of early-onset colon cancer. He undergoes colonoscopy, which identifies hundreds of polyps. Which gene is most likely involved?"
choices = [
"1. APC",
"2. EPCAM",
"3. MSH6",
"4. PTEN"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
At his annual check-up, a young adult mentions a strong family history of early-onset colon cancer. He undergoes colonoscopy, which identifies hundreds of polyps. Which gene is most likely involved?
APC
EPCAM
MSH6
PTEN
[question End]

Based on the information provided, the gene most likely involved in the patient'<s>


### 406

In [67]:
question = "Which term refers to a representation of all the different versions of genomic information within a species?"
choices = [
"1. Diploid genome",
"2. Exome",
"3. Haplotype",
"4. Pangenome"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Which term refers to a representation of all the different versions of genomic information within a species?
1. Diploid genome
2. Exome
3. Haplotype
4. Pangenome
[question End]

The term "pan genome" refers to a representation of all the different versions of genomic information within a species.</s><s>


### 407

In [68]:
question = "You are evaluating a patient, and suspect a certain diagnosis based on history and examination. You order exome sequencing. A diagnosis of which condition would most likely be molecularly confirmed this way?"
choices = [
"1. Bardet-Biedl syndrome",
"2. Beckwith-Wiedemann syndrome",
"3. Fragile X syndrome",
"4. Prader-Willi syndrome"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
You are evaluating a patient, and suspect a certain diagnosis based on history and examination. You order exome sequencing. A diagnosis of which condition would most likely be molecularly confirmed this way?
1. Bardet-Biedl syndrome
2. Beckwith-Wiedemann syndrome
3. Fragile X syndrome
4. Prader-Willi syndrome
[question End]

Based on the information provided, the most likely condition to be molecularly confirmed through exome sequencing is Fragile X syndrome. This is because Fragile X syndrome is the most common genetic cause of autism and intellectual disability, and exome sequencing is a common diagnostic tool for identifying genetic causes of these conditions

### 408

In [71]:
question = "A man has bi-allelic pathogenic variants in the CFTR gene, but does not have clinical signs of cystic fibrosis. He may have congenital absence of which of the following?"
choices = [
"1. Neutrophils",
"2. Pancreas",
"3. Thymus",
"4. Vas deferens"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A man has bi-allelic pathogenic variants in the CFTR gene, but does not have clinical signs of cystic fibrosis. He may have congenital absence of which of the following?
1. Neutrophils
2. Pancreas
3. Thymus
4. Vas deferens
[question End]

The man has bi-allelic pathogenic variants in the CFTR gene, but does not have clinical signs of cystic fibrosis. Therefore, he may have congenital absence of the vas deferens, but it is not possible to determine the absence of neutrophils or the thymus since they are not mentioned in the given information.</s><s>


### 409

In [72]:
question = " A pregnant person undergoes non-invasive prenatal screening (NIPS), which shows a result of 45,X for the pregnancy. According to the literature, what is the approximate positive predictive value of this result (the likelihood that this result is a true positive)?"
choices = [
"1. 5-7%",
"2. 20-30%",
"3. 50-70%",
"4. 97-99%"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
 A pregnant person undergoes non-invasive prenatal screening (NIPS), which shows a result of 45,X for the pregnancy. According to the literature, what is the approximate positive predictive value of this result (the likelihood that this result is a true positive)?
1. 5-7%
2. 20-30%
3. 50-70%
4. 97-99%
[question End]

The approximate positive predictive value of the result 45,X for non-invasive prenatal screening is 50-70%, according to the literature.</s><s>


### 410

In [74]:
question = "A neonate has progressive encephalopathy in the first week of life. Plasma amino acids show increased levels of leucine, isoleucine, valine, and alloisoleucine. What is the most likely diagnosis?"
choices = [
"1. Biotinidase deficiency",
"2. Homocystinuria",
"3. Maple syrup urine disease",
"4. Phenylketonuria"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A neonate has progressive encephalopathy in the first week of life. Plasma amino acids show increased levels of leucine, isoleucine, valine, and alloisoleucine. What is the most likely diagnosis?
1. Biotinidase deficiency
2. Homocystinuria
3. Maple syrup urine disease
4. Phenylketonuria
[question End]

Based on the information provided, the most likely diagnosis for the neonate's progressive encephalopathy is biotinidase deficiency. This is supported by the increased levels of leucine, isoleucine, valine, and alloisoleucine in the plasma amino acids. Biotinidase deficiency is a rare genetic disorder that can lead to neurological symptoms such as encephalopathy.<

### 411

In [75]:
question = "Third-trimester ultrasound shows an increased (>95th %) head circumference, slightly increased abdominal circumference, and long bones below the third percentile for gestational age. What condition is most likely?"
choices = [
"1. Achondroplasia",
"2. Arthogryposis",
"3. Osteopetrosis",
"4. Stickler syndrome"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Third-trimester ultrasound shows an increased (>95th %) head circumference, slightly increased abdominal circumference, and long bones below the third percentile for gestational age. What condition is most likely?
1. Achondroplasia
2. Arthogryposis
3. Osteopetrosis
4. Stickler syndrome
[question End]

Based on the information provided in the third-trimester ultrasound, the most likely condition is achondroplasia. This is a genetic disorder that results in the formation of short, stubby limbs with disproportionately large heads and is commonly associated with intellectual disability.</s><s>


### 412

In [76]:
question = "A person has been diagnosed with Von Hippel-Lindau disease. Overall, what is the common type of neoplasm in people with this condition?"
choices = [
"1. Endolymphatic sac tumor",
"2. Hemangioblastoma",
"3. Pheochromocytoma",
"4. Renal cell carcinoma"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A person has been diagnosed with Von Hippel-Lindau disease. Overall, what is the common type of neoplasm in people with this condition?
1. Endolymphatic sac tumor
2. Hemangioblastoma
3. Pheochromocytoma
4. Renal cell carcinoma
[question End]

The common type of neoplasm in individuals with Von Hippel-Lindau disease is renal cell carcinoma, according to the given information in the question.</s><s>


### 413

In [77]:
question = "A person enrolled in a research study is suspected to have a genetic condition. You use a multi-layered neural network to analyze photos of the person in order to aid the differential diagnosis. What is the best description of this type of artificial intelligence approach?"
choices = [
"1. Bayesian inference",
"2. Deep learning",
"3. Diffusion model",
"4. Unsupervised learning"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A person enrolled in a research study is suspected to have a genetic condition. You use a multi-layered neural network to analyze photos of the person in order to aid the differential diagnosis. What is the best description of this type of artificial intelligence approach?
1. Bayesian inference
2. Deep learning
3. Diffusion model
4. Unsupervised learning
[question End]

Based on the information provided, the best description of the approach used is deep learning. This type of artificial intelligence technique involves training a neural network to recognize patterns and make predictions based on large datasets. It is a powerful tool for analyzing images and other

### 415

In [78]:
question = "An infant is diagnosed with a severe form of beta-thalassemia. As part of the diagnostic work-up, what analyte would be expected to be absent or reduced on laboratory-based testing?"
choices = [
"1. Factor VIII",
"2. Hemoglobin A",
"3. Hemoglobin F",
"4. Platelets"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
An infant is diagnosed with a severe form of beta-thalassemia. As part of the diagnostic work-up, what analyte would be expected to be absent or reduced on laboratory-based testing?
1. Factor VIII
2. Hemoglobin A
3. Hemoglobin F
4. Platelets
[question End]

Based on the given information, it is expected that laboratory-based testing would show a reduction in the level of Hemoglobin F in the infant diagnosed with severe beta-thalassemia. However, there is no information provided in the question regarding the absence or reduction of any other analyte, such as Factor VIII or Platelets, which makes it unanswerable.</s><s>


### 416

In [79]:
question = "A person with a suspected genetic condition undergoes genomic testing, which identifies an explanatory, novel pathogenic variant in an established disease gene. The lab deposits the variant into a public database. Into which database was the variant most likely deposited?"
choices = [
"1. 1000 Genomes",
"2. ClinVar",
"3. GeneMatcher",
"4. gnomAD"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A person with a suspected genetic condition undergoes genomic testing, which identifies an explanatory, novel pathogenic variant in an established disease gene. The lab deposits the variant into a public database. Into which database was the variant most likely deposited?
1. 1000 Genomes
2. ClinVar
3. GeneMatcher
4. gnomAD
[question End]

The variant was most likely deposited in the 1000 Genomes database.</s><s>


### 418

In [81]:
question = "In general (and with the understanding that there can be exceptions and atypical, rare situations), which of the following genetic changes would most likely be suspected to affect canonical splicing?"
choices = [
"1. c.1845+1G>A",
"2. c.888+651T>A",
"3. c.3362A>G (p.Glu1121Gly)",
"4. c.413-24A>G "
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
In general (and with the understanding that there can be exceptions and atypical, rare situations), which of the following genetic changes would most likely be suspected to affect canonical splicing?
1. c.1845+1G>A
2. c.888+651T>A
3. c.3362A>G (p.Glu1121Gly)
4. c.413-24A>G 
[question End]

Based on the information provided, the most likely genetic change that would affect canonical splicing would be c.3362A>G (p.Glu1121Gly). This change results in a premature stop codon and could lead to the truncation of the mRNA and subsequent loss of protein function. The other changes mentioned are less likely to affect splicing as they do not result in a change in the codin

### 421

In [82]:
question = "A research laboratory in the United States supports a genomic research program. The lab decides to obtain CLIA certification. This certification is most important to allow the lab to be able to do which of the following:"
choices = [
"1. Bill for services",
"2. Conduct sequencing",
"3. Return individual results",
"4. Study children"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A research laboratory in the United States supports a genomic research program. The lab decides to obtain CLIA certification. This certification is most important to allow the lab to be able to do which of the following:
1. Bill for services
2. Conduct sequencing
3. Return individual results
4. Study children
[question End]

The CLIA certification is important for the lab to be able to bill for services, conduct sequencing, return individual results, and study children.</s><s>


### 425

In [83]:
question = "Out of 100 people with pathogenic variants in a certain gene, 50 develop heart disease; out of 100 people without pathogenic variants in that gene, 10 develop heart disease. What is the relative risk of heart disease in people with pathogenic variants in that gene?"
choices = [
"1. 0.4",
"2. 0.5",
"3. 1",
"4. 5"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
Out of 100 people with pathogenic variants in a certain gene, 50 develop heart disease; out of 100 people without pathogenic variants in that gene, 10 develop heart disease. What is the relative risk of heart disease in people with pathogenic variants in that gene?
1. 0.4
2. 0.5
3. 1
4. 5
[question End]

The relative risk of heart disease in people with pathogenic variants in that gene is 0.4, which is lower than the relative risk of heart disease in people without pathogenic variants in that gene (0.5).</s><s>


### 426

In [84]:
question = "The wings of a bat and a bird have similar forms and functions but did not develop in this way due to a common ancestor. What is the best term for the relationship between these wing structures?"
choices = [
"1. Analogous",
"2. Divergent",
"3. Embryologic",
"4. Homologous"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
The wings of a bat and a bird have similar forms and functions but did not develop in this way due to a common ancestor. What is the best term for the relationship between these wing structures?
1. Analogous
2. Divergent
3. Embryologic
4. Homologous
[question End]

The term "homologous" is the most appropriate for the relationship between the wings of a bat and a bird, as they both evolved from a common ancestor and share similar structures and functions.</s><s>


### 427

In [85]:
question = "You are analyzing whole genome sequencing data for patients with suspected  genetic conditions. For which of the following conditions would it be logical to analyze the most number of known disease-related genes to identify the molecular cause?"
choices = [
"1. Cornelia de Lange syndrome",
"2. Neurofibromatosis type 1",
"3. Propionic acidemia",
"4. Tuberous sclerosis"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
You are analyzing whole genome sequencing data for patients with suspected  genetic conditions. For which of the following conditions would it be logical to analyze the most number of known disease-related genes to identify the molecular cause?
1. Cornelia de Lange syndrome
2. Neurofibromatosis type 1
3. Propionic acidemia
4. Tuberous sclerosis
[question End]

Based on the given information, it would be most logical to analyze the most number of known disease-related genes for Cornelia de Lange syndrome since it is a rare genetic disorder that affects many parts of the body and can result in a wide range of symptoms. However, since the specific genes associated 

### 428

In [86]:
question = "A teen-ager has a several year history of intellectual deterioration, ataxia, and progressive myoclonic epilepsy. His father had a condition in middle age that included ataxia, dementia, progressive psychiatric disturbances, and choreoathetosis. What condition would be most likely?"
choices = [
"1. Ataxia-telangiectasia",
"2. Dentatorubral-pallidoluysian atrophy (DRPLA) ",
"3. Friedreich ataxia",
"4. Huntington disease"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
A teen-ager has a several year history of intellectual deterioration, ataxia, and progressive myoclonic epilepsy. His father had a condition in middle age that included ataxia, dementia, progressive psychiatric disturbances, and choreoathetosis. What condition would be most likely?
1. Ataxia-telangiectasia
2. Dentatorubral-pallidoluysian atrophy (DRPLA) 
3. Friedreich ataxia
4. Huntington disease
[question End]

Based on the information provided, the condition that would be most likely for the teen-ager and his father is Friedreich ataxia. This is a genetic condition that affects the nervous system and can cause progressive neurological symptoms such as ataxia, 

### 429

In [87]:
question = "In general (and with the caveat that there can be exceptions), a person affected by which of the following conditions would be expected to demonstrate overt clinical manifestations at the earliest age?"
choices = [
"1. Dravet syndrome",
"2. Hereditary hemochromatosis",
"3. Maturity Onset Diabetes of the Young (MODY)",
"4. Spinal and Bulbar Muscular Atrophy, X-linked 1 (Kennedy disease)"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
In general (and with the caveat that there can be exceptions), a person affected by which of the following conditions would be expected to demonstrate overt clinical manifestations at the earliest age?
1. Dravet syndrome
2. Hereditary hemochromatosis
3. Maturity Onset Diabetes of the Young (MODY)
4. Spinal and Bulbar Muscular Atrophy, X-linked 1 (Kennedy disease)
[question End]

Based on the given information, a person affected by Dravet syndrome would be expected to demonstrate overt clinical manifestations at the earliest age. This is because Dravet syndrome is a severe form of epilepsy that begins in the first few months of life and can cause developmental de

### 430

In [88]:
question = "You evaluate a young child with very convincing signs of Silver-Russell syndrome. Which test(s) would be most logical to order first?"
choices = [
"1. 11p15 methylation and UPD7 testing",
"2. Acylcarnitine profile",
"3. Karyotype and FISH for 22q11.2",
"4. Microarray"
]
query = question + "\n" + "\n".join(choices)
generate_answer(query)

<s> You are an intelligent genetic expert.
Below is a question related to the gene. 
Write an answer with detailed explanation.
Even if you are not sure about the answer, you should choose one choice from the options provided.
The response should provide an accurate answer to the question, while being concise.

[Question begin]
You evaluate a young child with very convincing signs of Silver-Russell syndrome. Which test(s) would be most logical to order first?
1. 11p15 methylation and UPD7 testing
2. Acylcarnitine profile
3. Karyotype and FISH for 22q11.2
4. Microarray
[question End]

Based on the information provided, the most logical test to order first would be 11p15 methylation and UPD7 testing. This test can help determine if the patient has a diagnosis of Silver-Russell syndrome, which is characterized by methylation abnormalities in the 11p15 region of the genome. However, if the patient has acylcarnitine profile results that are abnormal, this should be addressed first as it can

In [4]:
import torch

# 현재 Setup 되어있는 device 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))

Available devices  2
Current cuda device  1
NVIDIA GeForce RTX 3090


In [5]:
print('------------변경 후------------')
# GPU 할당 변경하기
GPU_NUM = 2 # 원하는 GPU 번호 입력
device = torch.device("cuda:1" if torch.cuda.is_available() else 'cpu')
print ('Current cuda device ', torch.cuda.current_device()) # check


------------변경 후------------
Current cuda device  1


In [6]:
!nvidia-smi

Mon Nov 13 02:00:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 30%   44C    P2   107W / 350W |   2344MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:41:00.0 Off |                  N/A |
|  0%   

In [6]:
import os
os.kill(654141, signal.SIGKILL)


NameError: name 'signal' is not defined

In [ ]:
!kill -9 654141